In [4]:
import random
from datetime import datetime, timedelta
from pymongo import MongoClient, GEO2D
from pymongo import MongoClient, GEOSPHERE

In [5]:
client = MongoClient("mongodb://localhost:27017/")
db = client["geo_monitoring"]
collection = db["displacement_data"]

In [10]:
collection.insert_one({
    "timestamp": datetime.now(),
    "sensor_id": "radar-1",
    "location": [116.4, 39.9],
    "displacement": {"x": 0.1, "y": -0.1, "z": 0.05}
})

InsertOneResult(ObjectId('68590453f5351b367a4a2522'), acknowledged=True)

In [8]:
list(collection.find())

[{'_id': ObjectId('6854e1617a5f4a8c266e7a1d'),
  'timestamp': datetime.datetime(2025, 6, 20, 4, 19, 45, 702000),
  'sensor_id': 'radar-4',
  'location': [116.44143577547453, 39.986699694458],
  'displacement': {'x': -0.571, 'y': 0.349, 'z': -0.25}},
 {'_id': ObjectId('6854e1617a5f4a8c266e7a1e'),
  'timestamp': datetime.datetime(2025, 6, 20, 4, 19, 45, 704000),
  'sensor_id': 'radar-4',
  'location': [116.4827615640738, 39.95973551086613],
  'displacement': {'x': -0.793, 'y': 0.583, 'z': -0.216}},
 {'_id': ObjectId('6854e1617a5f4a8c266e7a1f'),
  'timestamp': datetime.datetime(2025, 6, 20, 4, 19, 45, 705000),
  'sensor_id': 'radar-5',
  'location': [116.34647387982083, 39.8009684650996],
  'displacement': {'x': -0.848, 'y': 0.781, 'z': -0.865}},
 {'_id': ObjectId('6854e1617a5f4a8c266e7a20'),
  'timestamp': datetime.datetime(2025, 6, 20, 4, 19, 45, 707000),
  'sensor_id': 'radar-2',
  'location': [116.38851601829865, 39.9017162421652],
  'displacement': {'x': 0.954, 'y': 0.73, 'z': 0.474}

In [12]:
now = datetime.now()
since = now - timedelta(hours=1)

results = collection.find({"timestamp": {"$gte": since}})

In [13]:
for doc in results:
    print(doc)

{'_id': ObjectId('6858d4fff5c5fe2d0cdd9cbc'), 'timestamp': datetime.datetime(2025, 6, 23, 4, 15, 59, 854000), 'sensor_id': 'radar-1', 'location': [116.4, 39.9], 'displacement': {'x': 0.1, 'y': -0.1, 'z': 0.05}}
{'_id': ObjectId('68590453f5351b367a4a2522'), 'timestamp': datetime.datetime(2025, 6, 23, 0, 37, 55, 177000), 'sensor_id': 'radar-1', 'location': [116.4, 39.9], 'displacement': {'x': 0.1, 'y': -0.1, 'z': 0.05}}


In [14]:
now

datetime.datetime(2025, 6, 23, 0, 38, 12, 455805)

In [18]:
list(collection.find({"sensor_id": "radar-2"},
    {"_id": 0, "displacement": 1}
))

[{'displacement': {'x': 0.954, 'y': 0.73, 'z': 0.474}},
 {'displacement': {'x': 0.476, 'y': -0.993, 'z': 0.695}},
 {'displacement': {'x': 0.47, 'y': -0.823, 'z': 0.788}},
 {'displacement': {'x': 0.614, 'y': 0.235, 'z': 0.319}},
 {'displacement': {'x': 0.198, 'y': 0.759, 'z': 0.24}},
 {'displacement': {'x': 0.824, 'y': 0.585, 'z': 0.003}},
 {'displacement': {'x': -0.095, 'y': -0.955, 'z': -0.777}},
 {'displacement': {'x': 0.235, 'y': -0.193, 'z': 0.743}},
 {'displacement': {'x': -0.194, 'y': -0.561, 'z': 0.248}},
 {'displacement': {'x': 0.352, 'y': 0.315, 'z': 0.065}},
 {'displacement': {'x': 0.773, 'y': 0.523, 'z': -0.196}},
 {'displacement': {'x': -0.667, 'y': -0.355, 'z': -0.606}},
 {'displacement': {'x': -0.705, 'y': -0.561, 'z': 0.535}},
 {'displacement': {'x': 0.255, 'y': -0.228, 'z': -0.805}},
 {'displacement': {'x': -0.119, 'y': -0.412, 'z': -0.886}},
 {'displacement': {'x': -0.043, 'y': 0.992, 'z': 0.498}},
 {'displacement': {'x': 0.898, 'y': 0.339, 'z': 0.562}},
 {'displacemen

In [19]:
collection.update_many(
    {"sensor_id": "radar-1"},
    {"$set": {"flagged": True}}
)

UpdateResult({'n': 36, 'nModified': 36, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [21]:
list(collection.find({
    "location": {
        "$near": {
            "$geometry": {"type": "Point", "coordinates": [116.4, 39.9]},
            "$maxDistance": 5000  # meters
        }
    }
}))

[{'_id': ObjectId('6858d4fff5c5fe2d0cdd9cbc'),
  'timestamp': datetime.datetime(2025, 6, 23, 4, 15, 59, 854000),
  'sensor_id': 'radar-1',
  'location': [116.4, 39.9],
  'displacement': {'x': 0.1, 'y': -0.1, 'z': 0.05},
  'flagged': True},
 {'_id': ObjectId('68590453f5351b367a4a2522'),
  'timestamp': datetime.datetime(2025, 6, 23, 0, 37, 55, 177000),
  'sensor_id': 'radar-1',
  'location': [116.4, 39.9],
  'displacement': {'x': 0.1, 'y': -0.1, 'z': 0.05},
  'flagged': True},
 {'_id': ObjectId('6854e1617a5f4a8c266e7a47'),
  'timestamp': datetime.datetime(2025, 6, 20, 4, 19, 45, 740000),
  'sensor_id': 'radar-1',
  'location': [116.3993580419008, 39.90467136743186],
  'displacement': {'x': 0.54, 'y': -0.4, 'z': -0.814},
  'flagged': True},
 {'_id': ObjectId('6854e1617a5f4a8c266e7a2b'),
  'timestamp': datetime.datetime(2025, 6, 20, 4, 19, 45, 719000),
  'sensor_id': 'radar-3',
  'location': [116.4076177208308, 39.90227737025765],
  'displacement': {'x': -0.377, 'y': -0.589, 'z': -0.041}},

In [22]:
from bson.son import SON

pipeline = [
    {"$match": {"sensor_id": "radar-3"}},
    {"$group": {
        "_id": "$sensor_id",
        "avg_z": {"$avg": "$displacement.z"}
    }}
]
list(collection.aggregate(pipeline))

[{'_id': 'radar-3', 'avg_z': -0.06511627906976744}]

In [23]:
from pymongo import MongoClient
from datetime import datetime, timedelta
from bson.son import SON

In [5]:
client = MongoClient("mongodb://localhost:27017/")
db = client["geo_monitoring"]
collection = db["displacement_data"]

In [24]:
def get_average_displacement(center_lon, center_lat, radius_m, minutes=60):
    cutoff_time = datetime.now() - timedelta(minutes=minutes)

    pipeline = [
        {
            "$match": {
                "timestamp": {"$gte": cutoff_time},
                "location": {
                    "$near": {
                        "$geometry": {
                            "type": "Point",
                            "coordinates": [center_lon, center_lat]
                        },
                        "$maxDistance": radius_m
                    }
                }
            }
        },
        {
            "$group": {
                "_id": None,
                "avg_x": {"$avg": "$displacement.x"},
                "avg_y": {"$avg": "$displacement.y"},
                "avg_z": {"$avg": "$displacement.z"},
                "count": {"$sum": 1}
            }
        }
    ]

    result = list(collection.aggregate(pipeline))
    if result:
        return {
            "sensor_count": result[0]["count"],
            "avg_displacement": {
                "x": round(result[0]["avg_x"], 3),
                "y": round(result[0]["avg_y"], 3),
                "z": round(result[0]["avg_z"], 3),
            }
        }
    else:
        return {
            "sensor_count": 0,
            "avg_displacement": {
                "x": 0.0,
                "y": 0.0,
                "z": 0.0
            }
        }